## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [214]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [215]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [216]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00,54639.00
mean,54837.87,5470190.03,11.36,2022.99,136.27,99.45,5.00,570.26,2.98
std,25487.44,2599806.93,7.02,0.08,183.87,133.16,5.33,18795.52,2.44
min,10033.00,1000886.00,1.00,2022.00,0.01,0.01,0.00,0.00,0.00
25%,33593.00,3196372.00,6.00,2023.00,20.80,14.84,1.45,32.00,1.00
50%,53305.00,5496966.00,11.00,2023.00,68.31,49.89,3.24,68.00,3.00
75%,78498.00,7733869.00,17.00,2023.00,173.16,125.57,6.89,170.00,5.00
max,99984.00,9998480.00,52.00,2023.00,1396.23,846.27,46.43,3958244.00,9.00


In [217]:
# Use this space to do any additional research
# and familiarize yourself with the data.


In [218]:
# What three item categories had the most entries?
df["category"].value_counts()[:3]

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [219]:
# For the category with the most entries, which subcategory had the most entries?
df[df["category"]=="consumables"]["subcategory"].value_counts().head(1)

subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [220]:
# Which five clients had the most entries in the data?
df["client_id"].value_counts()[:5]

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [221]:
# Store the client ids of those top 5 clients in a list.
df["client_id"].value_counts()[:5].index.to_list()

[33615, 66037, 46820, 38378, 24741]

In [222]:
# How many total units (the qty column) did the client with the most entries order order?
sum(df[df["client_id"] == 33615]["qty"].to_list())

64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [223]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']
df['line_subtotal'] = df['line_subtotal'].round(2)

# Group by 'line_number' and calculate the subtotal for each line
grouped_df = df.groupby('line_number').agg({
    'unit_price': 'first',  # Take the first (assuming it's consistent within line_number)
    'qty': 'sum',          # Sum up the quantities
    'line_subtotal': 'sum' # Sum up the line subtotals
}).reset_index()
grouped_df.head()

,line_number,unit_price,qty,line_subtotal
0,0,24.95,3910842,569320726.01
1,1,1096.80,8485683,1101509718.00
2,2,229.74,3606967,397941284.05
3,3,36.42,4079221,633599493.74
4,4,20.99,3298118,389872708.19


In [224]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
df["total_weight"] = df["unit_weight"] * df["qty"]
df['shipping_price'] = df['total_weight'].apply(
    lambda weight: weight * 7 if weight > 50 else weight * 10
).round(2)


In [225]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df['line_price'] = (df['line_subtotal'] + df["shipping_price"]) * 1.0925
df['line_price'] = df['line_price'].round(2)
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,wall art,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,pens,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,project management,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,pens,36.42,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,misc,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40


In [226]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df["line_cost"] = df['qty'] * df['unit_cost'] + df['shipping_price']
df['line_cost'] = df['line_cost'].round(2)


In [227]:
# Create a column for the profit of each line using line cost and line price
df['line_profit'] = df['line_price'] - df['line_cost']
df['line_profit'] = df['line_profit'].round(2)
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.80


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [228]:
# Check your work using the totals above
selected_order_ids = [2742071, 2173913, 6128929]
filtered_df = df[df['order_id'].isin(selected_order_ids)]
filtered_df[['order_id', 'line_price']].groupby('order_id').sum()


,line_price
order_id,
2173913,162388.71
2742071,152811.89
6128929,923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [229]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
pd.set_option('display.float_format', '{:.2f}'.format)
# Group by client_id and calculate the total quantity for each client
client_qty = df.groupby('client_id').agg(
    total_qty = ('qty', 'sum'),
    total_line_cost = ('line_cost', 'sum')

).reset_index()

# Sort the clients by total quantity in descending order and select the top 5
top_5_clients = client_qty.sort_values(by='total_qty',ascending=False).head(5)

top_5_clients[['client_id', 'total_line_cost']]


,client_id,total_line_cost
758,78965,408570827.59
850,86721,138291601.58
204,30143,22078463.94
840,85931,128122872.60
738,77354,59510391.67


In [230]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
client_qty = df.groupby('client_id').agg({
    'first': 'first',
    'last': 'first',
    'qty': 'sum',
    'shipping_price': 'sum',
    'line_cost': 'sum',
    'shipping_price': 'sum',
    'line_price': 'sum',
    'line_profit': 'sum'
    



}).reset_index()
top_5_clients = client_qty.sort_values(by='qty',ascending=False).head(5)
top_5_clients[['client_id', 'qty', 'shipping_price', 'line_price', 'line_profit']].head()

,client_id,qty,shipping_price,line_price,line_profit
758,78965,4033302,5340756.88,655923670.05,247352842.46
850,86721,1010426,64335727.72,181266297.05,42974695.47
204,30143,846069,8981905.24,33328849.85,11250385.91
840,85931,644423,10122549.19,237417001.41,109294128.81
738,77354,506382,23814685.86,71942651.81,12432260.14


In [231]:
# Format the data and rename the columns to names suitable for presentation.
# Rename columns to more legible names
client_qty.rename(columns={
    'client_id': 'Client ID',
    'first': 'First Name',
    'last': 'Last Name',
    'qty': 'Quantity',
    'shipping_price': 'Shipping',
    'line_cost': 'Cost',
    'shipping_price': 'Shipping',
    'line_price': 'Revenue',
    'line_profit': 'Profit'
}, inplace=True)



print(client_qty.columns)
# Define the money columns. 
money_columns = ['Shipping', 'Revenue', 'Cost', 'Profit']

# Define a function that converts a dollar amount to millions.
def millionconversion(dollar):
    return dollar/1_000_000

# Apply the currency_format_millions function to only the money columns. 
client_qty[money_columns].apply(millionconversion)



# Convert the values to millions and rename the columns
for col in money_columns:
    if col in client_qty.columns:  # Ensure the column exists in the DataFrame
        client_qty[col] =  client_qty[col].apply(millionconversion)
        client_qty.rename(columns={col: f"{col} (millions)"}, inplace=True)

# Rename the columns to reflect the change in the money format. 
client_qty.head()



Index(['Client ID', 'First Name', 'Last Name', 'Quantity', 'Shipping', 'Cost',
       'Revenue', 'Profit'],
      dtype='object')


,Client ID,First Name,Last Name,Quantity,Shipping (millions),Cost (millions),Revenue (millions),Profit (millions)
0,10033,David,Ochoa,3153,0.14,0.49,0.69,0.20
1,10091,Shannon,Jimenez,29711,0.56,1.23,1.83,0.59
2,10218,Jennifer,Allen,32559,0.48,3.03,3.68,0.65
3,10351,Deborah,Cook,7615,0.24,1.25,1.71,0.47
4,10435,Derek,Hale,36967,1.03,6.23,8.72,2.49


In [236]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
sorted_summary = client_qty.sort_values(by='Profit (millions)', ascending = False)
sorted_summary.head()

,Client ID,First Name,Last Name,Quantity,Shipping (millions),Cost (millions),Revenue (millions),Profit (millions)
758,78965,Julie,Anderson,4033302,5.34,408.57,655.92,247.35
840,85931,Dennis,Hawkins,644423,10.12,128.12,237.42,109.29
320,38473,Michele,Mcdonald,402770,4.39,140.20,183.55,43.35
850,86721,Amy,Stewart,1010426,64.34,138.29,181.27,42.97
144,24741,Kendra,Garrett,239862,5.13,45.69,82.27,36.58
